# Practical for AI4ER. Using Machine Learning for Segmenting Mineral Phases

In [2]:
#%matplotlib qt5
%matplotlib qt
import hyperspy.api as hs
import numpy as np
from numpy import mean, std, median, sqrt, exp, log, delete

import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler

import scipy
from scipy import ndimage as ndi
from scipy.stats import sem, t, norm, gaussian_kde, mstats, iqr
from scipy.optimize import curve_fit

import skimage as ski
from skimage.segmentation import random_walker
from skimage.feature import peak_local_max
from skimage import exposure
from skimage import measure
from skimage import morphology as mph
from skimage import restoration
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray

# Table of Contents <a id='link1'></a>

<a href='#link2'> **1.0 Loading and Saving data**</a>

<a href='#link3'> **2.0 Machine Learning for Separating Phases**</a>

<a href='#link6'> **2.1 Identifying the number of phases**</a>

<a href='#link7'> **2.2 Creating a phase mask**</a>

<a href='#link4'> **3.0 Quantification of Phases - Mineralogy**</a>

<a href='#link5'> **4.0 Quantification of Elements - Morphology**</a>

This notebook contains a commented workflow on running machine learning on EDS data for phase separation and quantifying morphology. Written by Matt Ball

## 1.0 Loading and Saving data <a id='link2'></a>

First we load in the EDS map data using the hyperspy load function and assign metadata to the object, depending on the file format of the data

### Loading .bcf data

In [3]:
a = hs.load('data/JM1_OL7_map1.bcf')
a

[<Signal2D, title: Ch 0, dimensions: (|1820, 1295)>,
 <Signal2D, title: Ch 0, dimensions: (|2000, 2000)>,
 <EDSSEMSpectrum, title: EDX, dimensions: (1820, 1295|2048)>]

In [4]:
s = a[2]
s.axes_manager.set_signal_dimension(1)

In [5]:
s.plot()

### Loading .rpl/.raw data

In [ ]:
s = hs.load('your_filename.rpl').as_signal1D(0)

In [ ]:
s.plot()

### Selecting a sub-section of the data

In [6]:
s_mini = s.data[0:200,500:800,:]
s_min = hs.signals.Signal1D(s_mini)
s_min.plot()

### Assigning metadata

In [7]:
s.set_signal_type("EDS_SEM")
s.axes_manager[-1].name = 'E'
s.axes_manager['E'].units = 'keV'
s.axes_manager['E'].scale = 0.01
#Set the offset, the difference between the measured peak location and the true location, often an
#artificial zero peak is added to EDS data which can be used for this. This may need to be done a few
#times to correctly set this value
s.axes_manager['E'].offset = 0
#Set the beam energy to the beam energy the EDS data was collected at in keV
s.metadata.Acquisition_instrument.SEM.beam_energy = 20

In [8]:
s_min.set_signal_type("EDS_SEM")
s_min.axes_manager[-1].name = 'E'
s_min.axes_manager['E'].units = 'keV'
s_min.axes_manager['E'].scale = 0.01
#Set the offset, the difference between the measured peak location and the true location, often an
#artificial zero peak is added to EDS data which can be used for this. This may need to be done a few
#times to correctly set this value
s_min.axes_manager['E'].offset = -0.3
#Set the beam energy to the beam energy the EDS data was collected at in keV
s_min.metadata.Acquisition_instrument.SEM.beam_energy = 20
s_min.plot()

In [9]:
s.plot()

### Saving data in the .hspy format (a variation on .hdf)

In [10]:
s.save('your_filename')

Once data has been saved as a .hspy format it can be quickly reloaded alongside all metadata previously attached

In [11]:
s = hs.load('your_filename.hspy')

<a href='#link1'> Return to Table of Contents</a>

## 2.0 Machine Learning for Separating Phases <a id='link3'></a>

### 2.1 Identifying the number of phases <a id='link6'></a>

First we run a PCA (Principal Component Analysis) on the data to identify the number of significant principal components. The different phases will consist of both a loading map and corresponding spectrum. These can be cycled through using the arrow keys. Also plotted are two scree plots, which show the percentage of the data which can be explained by each principal component, one of which is plotted with a logarithmic y-axis, due to the large amount of data, this will be too slow to perform on the full dataset, so we will instead use the subset of data we created earlier, whilst the results of the machine learning on the full set can be loaded in and viewed

In [12]:
s_min.change_dtype('float32')
s_min.decomposition(True, algorithm='svd', output_dimension=20)
s_min.plot_decomposition_results()
s_min.plot_explained_variance_ratio(log=False)
s_min.plot_explained_variance_ratio(log=True)

Traceback (most recent call last):
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/drawing/utils.py", line 172, in function_wrapper
    function()
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/drawing/signal1d.py", line 145, in _on_close
    super(Signal1DFigure, self)._on_close()
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/drawing/figure.py", line 107, in _on_close
    self.events.closed.trigger(obj=self)
  File "<string>", line 4, in trigger
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/events.py", line 402, in trigger
    function(**{kw: kwargs.get(kw, None) for kw in kwsl})
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/signal.py", line 2133, in <lambda>
    lambda: self.events.data_changed.disconnect(self.update_plo

It is worth noting that the results of this PCA contain both negative loading intensities as well as negative spectra, neither of which is physically meaningful. This first step is really to identify the number of significant phases which the machine learning can pull apart

Again we can save the results of this analysis

In [ ]:
s.learning_results.save('your_filename_PCA')

Here's one I made earlier! Now we can load in the PCA results on the whole dataset to compare to our subsection. Alternatively we can load in results of any previous PCA

In [13]:
s.learning_results.load('AI4ER_JM1_OL7_PCA.npz')

In [14]:
s.plot_decomposition_results()
s.plot_explained_variance_ratio(log=False)
s.plot_explained_variance_ratio(log=True)

### Defining the number of principal components

The number of principal components can be defined either by the number which is below a critical value of variance or by the number which seem visually important

In [17]:
PC = 4

We now use this number of principal components to run a non-negative matrix factorisation, a clustering algorithm which produces only positive loadings and spectra, removing the issue of non physically meaningful results. Again we will perform this only on the subset of the data and load in a previously collected result for the full dataset

In [18]:
s_min.decomposition(True, algorithm='nmf', output_dimension=PC)
s_min.plot_decomposition_results()

Again we can save the results of this analysis

In [ ]:
s.learning_results.save('your_filename_NMF')

And again here's another pre-NMF-ed set of results. We could alternatively load in other previously collected results

In [19]:
s.learning_results.load('AI4ER_JM1_OL7_NMF.npz')

In [20]:
s.plot_decomposition_results()

### 2.2 Creating a phase mask <a id='link7'></a>

We can now use the results of the NMF to create phase masks which we will use to extract phase specific morphological data

In [22]:
pmap = s.get_decomposition_loadings()
mapsize = pmap.data.shape
phase_map = pmap.deepcopy()
k = pmap.deepcopy()
k.plot()

We use an otsu filter (which finds the ideal value to separate pixels based on an assumed bimodal histogram of pixel intensities) to select pixels within each phase map which correspond to the phase rather than the background and use these to create a set of binary phase maps

In [23]:
phase_otsu = np.zeros(len(phase_map), dtype='float32')

for q in range (len(phase_map)):
    phase_otsu[q-1] = ski.filters.threshold_otsu(k.inav[q-1].data)
    phase_map.inav[q-1].data[k.inav[q-1].data < phase_otsu[q-1]] = 0
    phase_map.inav[q-1].data[k.inav[q-1].data >= phase_otsu[q-1]] = 1
phase_map.plot()

We then apply two filters, one to remove small objects and one to remove small holes. The filters rely on the parameter 'minsize', which is defined as the minimum number of pixels an object or hole must contain before it is considered a real feature of the mask. The next three lines may need to be run with different values of 'minsize' in order to create an adequate mask

In [28]:
minsize = 100

In [29]:
phase_map.change_dtype('bool')
for q in range (len(phase_map)):
    phase = phase_map.inav[q]
    phase.data = mph.remove_small_objects(phase.data, min_size=minsize)
    phase_map.inav[q] = phase
    phase1 = phase_map.inav[q]
    phase1.data = mph.remove_small_holes(phase1.data, min_size=minsize)
    phase_map.inav[q] = phase1

mask = phase_map
mask.plot()

/space/maclenna/miniconda3/lib/python3.7/site-packages/skimage/morphology/misc.py:211: UserWarning: the min_size argument is deprecated and will be removed in 0.16. Use area_threshold instead.
  warn("the min_size argument is deprecated and will be removed in " +


<a href='#link1'> Return to Table of Contents</a>

## 3.0 Quantification of Phases - Mineralogy <a id='link4'></a>

In order to define the mineralogy of different phases (and which phases represent distinct mineral phases) we need to sum the signal of the original data over the mineral phase masks and plot these sum spectra

In [30]:
phase_spectrum = np.zeros((PC,mapsize[1],mapsize[2],2048))
phase_spectrum = hs.signals.Signal2D(phase_spectrum)
phase_spectrum.axes_manager.set_signal_dimension(3)

for q in range (len(phase_map)):
    m = mask.inav[q]
    m.axes_manager.set_signal_dimension(0)
    phase_spectrum.inav[q] = m * s

phase_spectrum.axes_manager.set_signal_dimension(1)
phase_spectrum.change_dtype('float32')

spectra_sum = np.zeros((len(phase_map),2048))
for q in range(len(phase_map)):
    spectra_sum[q-1] = phase_spectrum.inav[:,:,q-1].sum()

spectra_sum = hs.signals.EDSSEMSpectrum(spectra_sum)
spectra_sum.plot()

MemoryError: Unable to allocate array with shape (4, 1295, 1820, 2048) and data type float64

Traceback (most recent call last):
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/drawing/utils.py", line 172, in function_wrapper
    function()
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/drawing/figure.py", line 107, in _on_close
    self.events.closed.trigger(obj=self)
  File "<string>", line 4, in trigger
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/events.py", line 402, in trigger
    function(**{kw: kwargs.get(kw, None) for kw in kwsl})
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/signal.py", line 2133, in <lambda>
    lambda: self.events.data_changed.disconnect(self.update_plot),
  File "/space/maclenna/miniconda3/lib/python3.7/site-packages/hyperspy/events.py", line 373, in disconnect
    (function, self))
ValueError: The <bound meth

We now know which mineral phase masks correspond to which real mineralogy, alternatively, we could reconstruct the original data from the machine learning output to remove noise and use this for more traditional 'guesswork'

In [27]:
mod = s.get_decomposition_model()
mod.plot()

MemoryError: Unable to allocate array with shape (2048, 2356900) and data type float64

<a href='#link1'> Return to Table of Contents</a>

## 4.0 Quantification of Phases  - Morphology <a id='link5'></a>

First we define some functions for quantifying and plotting morphological data, from the 'grain_size_tools' toolbox

In [ ]:
def area2diameter(areas, correct_diameter=None):
    """ Calculate the equivalent cirular diameter from sectional areas.
    Parameters
    ----------
    areas : array_like
        the sectional areas of the grains
    correct_diameter : None or positive scalar, optional
        add the width of the grain boundaries to correct the diameters. If
        correct_diameter is not declared no correction is considered.
    Returns
    -------
    A numpy array with the equivalent circular diameters
    """

    # calculate the equivalent circular diameter
    diameters = 2 * np.sqrt(areas / np.pi)

    # diameter correction adding edges (if applicable)
    if correct_diameter is not None:
        diameters += correct_diameter

    return diameters

def freq_plot(diameters, binList, xgrid, y_values, y_max, x_peak, mean_GS, median_GS, plot, gmean=None):
    """ Generate a frequency vs grain size plot"""

    fig, ax = plt.subplots()

    ax.hist(diameters,
            bins=binList,
            range=(0, diameters.max()),
            density=True,
            color='#80419d',
            edgecolor='#C59fd7',
            alpha=0.7)
    ax.plot([mean_GS, mean_GS], [0, y_max],
            linestyle='-',
            color='#2F4858',
            label='arith. mean',
            linewidth=2.5)
    ax.plot([median_GS, median_GS], [0, y_max],
            linestyle='--',
            color='#2F4858',
            label='median',
            linewidth=2.5)

    ax.set_ylabel('density', color='#252525')

    if plot == 'linear':
        ax.plot([gmean, gmean], [0, y_max],
                linestyle='-',
                color='C1',
                label='geo. mean')
        ax.set_xlabel(r'apparent diameter ($\mu m$)', color='#252525')

    elif plot == 'log':
        ax.set_xlabel(r'apparent diameter $\log_e{(\mu m)}$', color='#252525')

    elif plot == 'log10':
        ax.set_xlabel(r'apparent diameter $\log_{10}{(\mu m)}$', color='#252525')

    elif plot == 'norm':
        ax.set_xlabel(r'normalized apparent diameter $\log_e{(\mu m)}$', color='#252525')

    elif plot == 'sqrt':
        ax.set_xlabel(r'Square root apparent diameter ($\sqrt{\mu m}$)', color='#252525')

    ax.plot(xgrid, y_values,
            color='#2F4858')

    ax.vlines(x_peak, 0, y_max,
              linestyle=':',
              color='#2F4858',
              label='kde peak',
              linewidth=2.5)

    ax.legend(loc='best', fontsize=16)
    ax.set_ylim(bottom=-0.001)

    fig.tight_layout()

    return plt.show()

def gen_xgrid(start, stop, precision):
    """ Returns a mesh of values (i.e. discretize the
    sample space) with a range and desired precision.
    Parameters
    ----------
    start : scalar
        the starting value of the sequence
    stop : scalar
        the end value of the sequence
    precision : scalar
        the desired precision (density) of the mesh
    """

    rango = stop - start

    # num = range / precision; as long as range > precision
    if rango < precision:
        raise ValueError('Caution! the precision must be smaller than the range of grain sizes')
    else:
        n = int(round(rango / precision, 0))

    return np.linspace(start, stop, num=n)

def calc_freq_peak(diameters, bandwidth, max_precision):
    """ Estimate the peak of the frequency ("mode") of a continuous
    distribution based on the Gaussian kernel density estimator. It
    uses Scipy's gaussian kde method.
    Parameters
    ----------
    diameters : array_like
        the diameters of the grains
    bandwidth : string, positive scalar or callable
        the method to estimate the bandwidth or a scalar directly defining the
        bandwidth. Methods can be 'silverman' or 'scott'.
    max_precision : positive scalar
        the maximum precision expected for the "peak" estimator.
    Call functions
    --------------
    - gen_xgrid
    - kde (from scipy)
    Returns
    -------
    The x and y values to contruct the kde, the peak grain size,
    the maximum density value,, and the bandwidth
    """

    # check bandwidth and estimate Gaussian kernel density function
    if isinstance(bandwidth, (int, float)):
        bw = bandwidth / diameters.std(ddof=1)
        kde = gaussian_kde(diameters, bw_method=bw)

    elif isinstance(bandwidth, str):
        kde = gaussian_kde(diameters, bw_method=bandwidth)
        bw = round(kde.covariance_factor() * diameters.std(ddof=1), 2)

    else:
        raise ValueError("bandwidth must be integer, float, or plug-in methods 'silverman' or 'scott'")

    # locate the peak
    xgrid = gen_xgrid(diameters.min(), diameters.max(), max_precision)
    densities = kde(xgrid)
    y_max, peak_grain_size = np.max(densities), xgrid[np.argmax(densities)]

    return xgrid, densities, peak_grain_size, y_max, bw


def calc_freq_grainsize(diameters, binsize, plot, bandwidth, max_precision):
    """ Calculate the distribution of grain sizes using the histogram and Gaussian
    kernel density estimator (KDE). It returns the modal interval, the middle value
    of modal interval, and the frequency peak based on the KDE, and call the
    function responsible for generating the corresponding plot.
    Parameters
    ----------
    diameters : array_like
        the diameters of the grains
    binsize : string (rule of thumb), or posive scalar
        the bin size
    plot : string
        the type of plot and grain size, either 'linear', 'log' or 'sqrt'.
    bandwidth : string, scalar or callable, optional
        the method to estimate the bandwidth or a scalar directly defining the
        bandwidth. Methods can be 'silverman' or 'scott'.
    max_precision : positive scalar
        the maximum precision expected for the "peak" kde-based estimator
    References
    ----------
    Scott, D.W. (1992) Multivariate Density Estimation: Theory, Practice, and Visualization
    Silverman, B.W. (1986) Density Estimation for Statistics and Data Analysis
    Call functions
    --------------
    - freq_plot
    - calc_freq_peak
    """

    if len(diameters) < 433:  # TODO: Change this in next version, this is not apply for all distributions!
        print(' ')
        print('Caution! You should use at least 433 grain measurements for reliable results')

    mean_GS, std_GS = mean(diameters), std(diameters)
    median_GS, iqr_GS = median(diameters), iqr(diameters)
    if plot == 'linear':
        gmean = mstats.gmean(diameters)  # geometric mean
        gsd = np.exp(np.std(np.log(diameters)))  # multiplicative (geometric) standard deviation
        mean_RMS = sqrt(mean(diameters**2))  # root mean square

    # estimate the number of classes using an automatic plug-in method (if apply)
    if type(binsize) is str:
        bin_method = binsize
        histogram, bin_edges = np.histogram(diameters, bins=binsize, range=(diameters.min(), diameters.max()))
        binsize = bin_edges[1] - bin_edges[0]
    else:
        bin_method = None
        bin_edges = np.arange(diameters.min(), diameters.max() + binsize, binsize)
        histogram, bin_edges = np.histogram(diameters, bins=bin_edges)

    # find the grain size range in which the histogram value is maximum
    modInt_leftEdge = bin_edges[np.argmax(histogram)]
    modInt_rightEdge = modInt_leftEdge + binsize

    # Estimate the frequency peak grain size based on kde
    x_kde, y_kde, peak, y_max, bw = calc_freq_peak(diameters, bandwidth, max_precision)

    print(' ')
    print('CENTRAL TENDENCY ESTIMATORS')
    print('Arithmetic mean = {} microns' .format(round(mean_GS, 2)))
    if plot == 'linear':
        print('Geometric mean = {} microns' .format(round(gmean, 2)))
        print('RMS mean = {} microns (discouraged)' .format(round(mean_RMS, 2)))
    print('Median = {} microns' .format(round(median_GS, 2)))
    print('Peak grain size (based on KDE) = {} microns' .format(round(peak, 2)))

    print(' ')
    print('DISTRIBUTION FEATURES (SPREADING AND SHAPE)')
    print('Standard deviation = {} (1-sigma)' .format(round(std_GS, 2)))
    print('Interquartile range (IQR) = {}' .format(round(iqr_GS, 2)))
    if plot == 'linear':
        print('Multiplicative standard deviation (lognormal shape) = {}' .format(round(gsd, 2)))

    print(' ')
    print('HISTOGRAM AND KDE FEATURES')
    print('The modal interval is {left} - {right}' .format(left=round(modInt_leftEdge, 2), right=round(modInt_rightEdge, 2)))
    print('The number of classes are {}' .format(len(histogram)))
    if type(bin_method) is str:
        print('The bin size is {bin} according to the {rule} rule' .format(bin=round(binsize, 2), rule=bin_method))

    if type(bandwidth) is str:
        print('KDE bandwidth = {a} ({b} rule)' .format(a=bw, b=bandwidth))
    else:
        print('KDE bandwidth =', bandwidth)

    if plot == 'linear':
        print('Maximum precision of the KDE estimator =', max_precision)
        return freq_plot(diameters, bin_edges, x_kde, y_kde, y_max, peak, mean_GS, median_GS, plot, gmean)
    elif plot == 'log':
        print('Maximum precision of the KDE estimator =', max_precision)
    elif plot == 'log10':
        print('Maximum precision of the KDE estimator =', max_precision)
    elif plot == 'sqrt':
        print('Maximum precision of the KDE estimator =', max_precision)

    return freq_plot(diameters, bin_edges, x_kde, y_kde, y_max, peak, mean_GS, median_GS, plot)

### Plotting grain size distributions

We now seperate grains which are touching but separate from each other using a watershed algorithm and subsequently calculate the area of the separated grains and the diameter of equivalent area spheres, which are saved off in .txt files

In [ ]:
mask.change_dtype('int')
for q in range(PC):
    distance = ndi.distance_transform_edt(mask.data[q])
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=mask.data[q])
    markers = ski.morphology.label(local_maxi)
    labels_ws = ski.morphology.watershed(-distance, markers, mask=mask.data[q])
    properties = ski.measure.regionprops(labels_ws)
    diameters = np.asarray([prop.equivalent_diameter for prop in properties])
    np.savetxt('AI4ER_diameters_'+str(q)+'.txt', diameters)


We can then read in the diameters of the different grains from the files and plot the distribution of grain sizes

In [ ]:
diameters = np.loadtxt('AI4ER_diameters_1.txt')
calc_freq_grainsize(diameters, binsize=0.1, plot='linear', bandwidth='silverman', max_precision=0.1)

<a href='#link1'> Return to Table of Contents</a>